In [ ]:
!git clone https://github.com/stuniy/SPO_PGU.git

Cloning into 'ML_School'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 56 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), done.


# Ансамбли. Стекинг
Еще один вариант ансамблей - стекинг - когда объединяются результаты разнородных моделей с помощью еще одной модели.
В `sklearn` реализованы 
* [`StackingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier)  - классификатор
* [`StackingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor) - регрессор

Аргументы для создания ансамблей: 
* `estimators` - кортеж из названий и объектов моделей ансамбля
* `final_estimator` - модель для объединения результатов, по умолчанию `LogisticRegression` для классификатора и `RidgeCV` для регрессора.
* `cv` - число разбиений для кроссвалидации (или объекты) 
stack_method - по какому именно результату объядинять модели: ‘auto’, ‘predict_proba’, ‘decision_function’, ‘predict’,
* `verbose` - число, определяющее как подробно выводить информацию об обучении, по умолчанию 0 - не выводить
* `passthrough` - использовать ли для обучения финальной модели сами данные или только результаты моделей ансамбля.

Создаваемые модели имеют атрибуты 

* `estimators_` - список обученных моделей в ансамбле
* `final_estimator_` - обученная финальная модель
* `named_estimators_` - контейнер для доступа к параметрам моделей по их названию
* `classes_` - для классификатора метки классов

Реализованные методы аналогичны: `fit()`, `pedict()`, ... 

Реализованы также  [`VotingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier) и [`VotingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html#sklearn.ensemble.VotingRegressor), отличающиеся тем, что в качестве финальной модели используется процедура голосования.



## Классификатор голосования
Идея `VotingClassifier` состоит в том, чтобы объединить концептуально разные классификаторы машинного обучения и использовать большинство голосов или средние предсказанные вероятности (мягкое голосование) для прогнозирования меток классов. Такой классификатор может быть полезен для набора одинаково хорошо работающих моделей, чтобы уравновесить их индивидуальные недостатки.

## Ярлыки класса большинства (большинство / жесткое голосование)
При голосовании большинством прогнозируемая метка класса для конкретной выборки является меткой класса, которая представляет большинство (режим) меток класса, предсказываемых каждым отдельным классификатором.

Например, если прогноз для данной выборки

классификатор 1 -> класс 1

классификатор 2 -> класс 1

классификатор 3 -> класс 2


Классификатор `VotingClassifier (с voting='hard')` классифицирует образец как «класс 1» на основе метки класса большинства.

В случае совпадения `VotingClassifier` класс будет выбран в соответствии с возрастающим порядком сортировки. Например, в следующем сценарии

классификатор 1 -> класс 2

классификатор 2 -> класс 1

метка класса 1 будет присвоена образцу.

В следующем примере показано, как соответствовать классификатору правил большинства:


In [11]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

iris = datasets.load_iris()
X, y = iris.data[:, 1:3], iris.target

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(
     estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
     voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'GaussianNB', 'Ensemble']):
     scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


Accuracy: 0.95 (+/- 0.04) [Logistic Regression]
Accuracy: 0.94 (+/- 0.04) [Random Forest]
Accuracy: 0.91 (+/- 0.04) [GaussianNB]
Accuracy: 0.95 (+/- 0.04) [Ensemble]


In [6]:
clf.estimators

[('lr', LogisticRegression(random_state=1)),
 ('rf', RandomForestClassifier(n_estimators=50, random_state=1)),
 ('gnb', GaussianNB())]

## Средневзвешенные вероятности (мягкое голосование)
В отличие от голосования большинством (жесткое голосование) мягкое голосование возвращает метку класса как argmax суммы предсказанных вероятностей.


Каждому классификатору можно присвоить определенные веса с помощью weights параметра. Когда веса предоставлены, прогнозируемые вероятности классов для каждого классификатора собираются, умножаются на вес классификатора и усредняются. Окончательная метка класса затем получается из метки класса с наивысшей средней вероятностью.


Чтобы проиллюстрировать это на простом примере, предположим, что у нас есть 3 классификатора и 3 задачи классификации, в которых мы присваиваем всем классификаторам одинаковые веса: w1 = 1, w2 = 1, w3 = 1.

Затем средневзвешенные вероятности для выборки будут рассчитаны следующим образом:

классификатор	1 класс	2 класс	3 класс

классификатор 1	w1 * 0,2	w1 * 0,5	w1 * 0,3

классификатор 2	w2 * 0,6	w2 * 0,3	w2 * 0,1

классификатор 3	w3 * 0,3	w3 * 0,4	w3 * 0,3

средневзвешенное	0,37	0,4	0,23

Здесь метка предсказанного класса равна 2, так как она имеет самую высокую среднюю вероятность.

В следующем примере показано, как области решений могут измениться при использовании программного обеспечения `VotingClassifier` на основе линейной машины опорных векторов, дерева решений и классификатора K-ближайших соседей:


In [10]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier

# Loading some example data
iris = datasets.load_iris()
X = iris.data[:, [0, 2]]
y = iris.target

# Training classifiers
clf1 = DecisionTreeClassifier(max_depth=4)
clf2 = KNeighborsClassifier(n_neighbors=7)
clf3 = SVC(kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)],
                         voting='soft', weights=[2, 1, 2])

clf1 = clf1.fit(X, y)
clf2 = clf2.fit(X, y)
clf3 = clf3.fit(X, y)
eclf = eclf.fit(X, y)

for clf, label in zip([clf1, clf2, clf3, eclf], ['DecisionTree', 'KNeighborst', 'SVC', 'VotingClassifier']):
     scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.95 (+/- 0.03) [DecisionTree]
Accuracy: 0.94 (+/- 0.04) [KNeighborst]
Accuracy: 0.95 (+/- 0.03) [SVC]
Accuracy: 0.95 (+/- 0.03) [VotingClassifier]


## Домашнее задание

1. Повторите эксперимент, лабораторной работы.
2. Используя один из наборов данных, например, diabets проведите классификацию.
3. Оценить качество построенной модели, основываясь на матрице неточности и ROC-анализе.
4. Посмотрите на коэффициенты логистической регрессии финальной модели и скажите, результаты какого классификатора из ансамбля более важны?
5. Пробуйте самостоятельно другие данные, другие модели, другие параметры моделей.